# KNN VS OLS REGRESSION
   This to create and compare KNN  and ols regression

  I will use the house prices dataset with selected features to run the ols regression andknn regression and compare the predictions.

In [5]:
import pandas as pd
from sqlalchemy import create_engine
import warnings

postgres_host = '142.93.121.174'
postgres_port = '5432'
postgres_db = 'houseprices'

In [6]:
engine2 = create_engine('postgresql://{}:{}@{}:{}/{}'.format(
    postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db))

houseprices_df = pd.read_sql_query('select * from houseprices',con=engine2)
engine2.dispose()

In [8]:
# select features  year build, sqfeet , lot area , full bath ,halfbath, overallcond, live area
fea_df = houseprices_df[['yearbuilt','lotarea','fullbath','halfbath','grlivarea','overallcond','saleprice']]

In [9]:
# bath and condition categorical
bath_df = pd.get_dummies(fea_df['fullbath'],dtype=int).rename(columns=lambda x:'fb_' +str(x))
new_df = fea_df.drop(['fullbath'], axis =1).merge(bath_df,left_index=True, right_index=True)
bath_df = pd.get_dummies(fea_df['halfbath'],dtype=int).rename(columns=lambda x:'hb_' +str(x))
new2_df = new_df.drop(['halfbath'], axis =1).merge(bath_df,left_index=True, right_index=True)

In [10]:
# `Y` is the target variable
Y = new2_df['saleprice']
# `X` is the feature set which includes the
# `is_male` and `is_smoker` variables
X = new2_df.drop(['saleprice'], axis =1)

In [11]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2, random_state = 465)

In [12]:
# build regression model
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from statsmodels.tools.eval_measures import mse, rmse
from sqlalchemy import create_engine
import numpy as np
import pandas as pd
# Fit an OLS model using scikit-learn
lrm = LinearRegression()
lrm.fit(X_train, y_train)


# You are making predictions here
y_preds_train = lrm.predict(X_train)
y_preds_test = lrm.predict(X_test)

print("R-squared of the model in the training set is: {}".format(lrm.score(X_train, y_train)))
print("-----Test set statistics-----")
print("R-squared of the model in the test set is: {}".format(lrm.score(X_test, y_test)))
print("Mean absolute error of the prediction is: {}".format(mean_absolute_error(y_test, y_preds_test)))
print("Mean squared error of the prediction is: {}".format(mse(y_test, y_preds_test)))
print("Root mean squared error of the prediction is: {}".format(rmse(y_test, y_preds_test)))
print("Mean absolute percentage error of the prediction is: {}".format(np.mean(np.abs((y_test - y_preds_test) / y_test)) * 100))

R-squared of the model in the training set is: 0.6925319399003738
-----Test set statistics-----
R-squared of the model in the test set is: 0.7078845618659644
Mean absolute error of the prediction is: 28069.088761753683
Mean squared error of the prediction is: 1961176608.5607944
Root mean squared error of the prediction is: 44285.17368782462
Mean absolute percentage error of the prediction is: 15.73929590479116


In [15]:
from sklearn import neighbors
# Run the same model, this time with weights
knn_w = neighbors.KNeighborsRegressor(n_neighbors=10, weights='distance')
knn_w.fit(X_train, y_train)



KNeighborsRegressor(n_neighbors=10, weights='distance')

In [17]:
from sklearn.model_selection import cross_val_score
score_w = cross_val_score(knn_w, X_train, y_train, cv=5)
print("Weighted Accuracy: %0.2f (+/- %0.2f)" % (score_w.mean(), score_w.std() * 2))
from sklearn.model_selection import cross_val_score
score_w = cross_val_score(knn_w, X_test, y_test, cv=5)
print("Weighted Accuracy: %0.2f (+/- %0.2f)" % (score_w.mean(), score_w.std() * 2))

Weighted Accuracy: 0.49 (+/- 0.10)
Weighted Accuracy: 0.41 (+/- 0.13)


In [18]:
score_w = cross_val_score(lrm, X_train, y_train, cv=5)
print("Weighted Accuracy: %0.2f (+/- %0.2f)" % (score_w.mean(), score_w.std() * 2))
score_w = cross_val_score(lrm, X_test, y_test, cv=5)
print("Weighted Accuracy: %0.2f (+/- %0.2f)" % (score_w.mean(), score_w.std() * 2))

Weighted Accuracy: 0.68 (+/- 0.16)
Weighted Accuracy: 0.63 (+/- 0.20)


# Summary
   from above OLS and KNN models the cross validation score for the OLS model is better compared to KNN regression. The dataset having multiple features and the prediction in OLS is more accurate compare to the KNN regression.